In [2]:
# Install necessary dependencies (run once)
!pip install geopandas pyogrio

# Imports
import pandas as pd
import geopandas as gpd

# === 1. Load ACS Demographic Data ===
acs_path = "ACSDP5Y2023.DP05-Data.csv"  # update if needed
acs = pd.read_csv(acs_path)

# Filter and rename
acs = acs[[
    'GEO_ID', 'NAME', 'DP05_0001E', 'DP05_0071PE', 'DP05_0077PE', 'DP05_0078PE', 'DP05_0033PE'
]].copy()
acs['FIPS'] = acs['GEO_ID'].str.extract(r'US(\d{5})')

acs.rename(columns={
    'DP05_0001E': 'Total_Pop',
    'DP05_0071PE': 'Pct_Black',
    'DP05_0077PE': 'Pct_Hispanic',
    'DP05_0078PE': 'Pct_White_NonHisp',
    'DP05_0033PE': 'Pct_Asian'
}, inplace=True)

# === 2. Load AHRF Workforce Data ===
ahrf_path = "ahrf2024_Feb2025.csv"  # update path if needed
ahrf = pd.read_csv(ahrf_path, low_memory=False)

# === 3. Load TIGER/Line County Shapefile ===
# First, unzip the shapefile if you uploaded it as a .zip:
# !unzip "/content/tl_2024_us_county.zip" -d /content/shapefile/




/tmp/ipython-input-2-2017903438.py:10: DtypeWarning: Columns (2,4,6,8,10,12,13,14,15,16,18,20,22,24,26,27,28,29,30,32,33,34,35,36,37,38,40,41,42,44,45,46,47,48,50,52,54,56,58,60,62,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,149,150,152,154,155,156,157,158,159,160,161,162,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,194,198,200,201,202,203,204,206,208,210,212,214,215,216,217,218,220,221,222,223,226,228,229,230,232,233,234,235,236,238,240,242,246,248,250,254,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,

In [3]:

# If files are already extracted:
shapefile_path = "tl_2024_us_county.shp"
gdf_counties = gpd.read_file(shapefile_path)

# Preview each
print("ACS:", acs.shape)

print("AHRF:", ahrf.shape)
print("Geo:", gdf_counties.shape)

ACS: (3223, 8)
AHRF: (3240, 4300)
Geo: (3235, 19)


In [4]:
print(acs.head())
print(ahrf.head())
print(gdf_counties.head())

           GEO_ID                     NAME  \
0       Geography     Geographic Area Name   
1  0500000US01001  Autauga County, Alabama   
2  0500000US01003  Baldwin County, Alabama   
3  0500000US01005  Barbour County, Alabama   
4  0500000US01007     Bibb County, Alabama   

                                 Total_Pop  \
0  Estimate!!SEX AND AGE!!Total population   
1                                    59285   
2                                   239945   
3                                    24757   
4                                    22152   

                                           Pct_Black  \
0  Percent!!Race alone or in combination with one...   
1                                                0.8   
2                                                1.6   
3                                                0.9   
4                                                1.9   

                                        Pct_Hispanic  \
0  Percent!!HISPANIC OR LATINO AND RACE!!Total po... 

In [5]:
#import libraries
!pip install plotly geopandas pyogrio seaborn -q

import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print(" All libraries imported successfully!")

 All libraries imported successfully!


In [6]:
#Quick Data Exploration

print("\n QUICK DATA EXPLORATION")
print("="*50)

# Check ACS data
print("\n ACS Data Sample:")
print(acs.head())
print(f"\nACS Columns: {list(acs.columns)}")

# Check AHRF columns for workforce data
print(f"\n AHRF Columns (first 20): {list(ahrf.columns[:20])}")
print(f"Total AHRF columns: {len(ahrf.columns)}")

# Check geographic data
print(f"\n Geographic Data Sample:")
print(gdf_counties[['GEOID', 'NAME', 'STATEFP', 'COUNTYFP']].head())


 QUICK DATA EXPLORATION

 ACS Data Sample:
           GEO_ID                     NAME  \
0       Geography     Geographic Area Name   
1  0500000US01001  Autauga County, Alabama   
2  0500000US01003  Baldwin County, Alabama   
3  0500000US01005  Barbour County, Alabama   
4  0500000US01007     Bibb County, Alabama   

                                 Total_Pop  \
0  Estimate!!SEX AND AGE!!Total population   
1                                    59285   
2                                   239945   
3                                    24757   
4                                    22152   

                                           Pct_Black  \
0  Percent!!Race alone or in combination with one...   
1                                                0.8   
2                                                1.6   
3                                                0.9   
4                                                1.9   

                                        Pct_Hispanic  \
0  Percen

In [7]:
# Data Cleaning
print("\n STEP-BY-STEP DATA CLEANING")
print("="*50)

# 1. Clean ACS data
print("\n1️⃣ Cleaning ACS demographic data...")
acs_clean = acs.iloc[1:].copy()  # Skip header row

# Convert to numeric
numeric_cols = ['Total_Pop', 'Pct_Black', 'Pct_Hispanic', 'Pct_White_NonHisp', 'Pct_Asian']
for col in numeric_cols:
    acs_clean[col] = pd.to_numeric(acs_clean[col], errors='coerce')

# Calculate minority percentage
acs_clean['Pct_Minority'] = (
    acs_clean['Pct_Black'].fillna(0) +
    acs_clean['Pct_Hispanic'].fillna(0) +
    acs_clean['Pct_Asian'].fillna(0)
)

# Clean FIPS
acs_clean['FIPS'] = acs_clean['FIPS'].str.zfill(5)
acs_clean = acs_clean.dropna(subset=['FIPS'])

print(f" ACS cleaned: {len(acs_clean)} counties")
print("Sample ACS cleaned data:")
print(acs_clean[['NAME', 'FIPS', 'Total_Pop', 'Pct_Minority']].head())


 STEP-BY-STEP DATA CLEANING

1️⃣ Cleaning ACS demographic data...
 ACS cleaned: 3222 counties
Sample ACS cleaned data:
                      NAME   FIPS  Total_Pop  Pct_Minority
1  Autauga County, Alabama  01001      59285       59287.6
2  Baldwin County, Alabama  01003     239945      239949.6
3  Barbour County, Alabama  01005      24757       24762.0
4     Bibb County, Alabama  01007      22152       22155.5
5   Blount County, Alabama  01009      59292       59302.9


In [8]:
# Extract Healthcare Workforce Data
print("\n Extracting healthcare workforce data...")

# Find workforce columns in AHRF
workforce_keywords = ['md_nf', 'rn_', 'dent_', 'psyc']
potential_cols = []
for keyword in workforce_keywords:
    potential_cols.extend([col for col in ahrf.columns if keyword in col.lower() and '22' in col])

print(f"Found {len(potential_cols)} potential workforce columns")
print("Sample workforce columns:", potential_cols[:10])

# Use specific columns we know exist
workforce_cols = {}

# Check what columns actually exist
if 'md_nf_22' in ahrf.columns:
    workforce_cols['md_nf_22'] = 'Total_Physicians'
elif 'MD_22' in ahrf.columns:
    workforce_cols['MD_22'] = 'Total_Physicians'

if 'rn_22' in ahrf.columns:
    workforce_cols['rn_22'] = 'Registered_Nurses'
elif 'RN_22' in ahrf.columns:
    workforce_cols['RN_22'] = 'Registered_Nurses'

if 'dent_22' in ahrf.columns:
    workforce_cols['dent_22'] = 'Dentists'

# Use general surgeons as available
surg_cols = [col for col in ahrf.columns if 'gen_surg' in col.lower() and '22' in col]
if surg_cols:
    print(f"Found surgeon columns: {surg_cols}")

print(f"Using workforce columns: {workforce_cols}")


 Extracting healthcare workforce data...
Found 901 potential workforce columns
Sample workforce columns: ['md_nf_prim_care_pc_excl_rsdnt_22', 'md_nf_prim_care_pc_rsdnt_22', 'md_nf_fed_activ_22', 'md_nf_activ_22', 'md_nf_fed_22', 'md_nf_22', 'md_nf_all_pc_22', 'md_nf_pc_ofc_22', 'md_nf_pc_hosp_all_22', 'md_nf_pc_rsdnt_22']
Found surgeon columns: ['md_nf_gen_surg_22', 'md_nf_gen_surg_all_pc_22', 'md_nf_gen_surg_pc_ofc_22', 'md_nf_gen_surg_pc_rsdnt_22', 'md_nf_gen_surg_pc_hosp_ft_22', 'md_nf_gen_surg_admin_22', 'md_nf_gen_surg_teach_22', 'md_nf_gen_surg_resrch_22', 'md_nf_gen_surg_oth_22', 'md_fed_gen_surg_pc_rsdnt_22', 'md_fed_gen_surg_pc_hosp_ft_22', 'md_fed_gen_surg_othprofl_22', 'md_nf_gen_surg_lt35_22', 'md_nf_gen_surg_35_44_22', 'md_nf_gen_surg_45_54_22', 'md_nf_gen_surg_55_64_22', 'md_nf_gen_surg_65_74_22', 'md_nf_gen_surg_ge75_22', 'do_nf_gen_surg_22', 'do_nf_gen_surg_all_pc_22']
Using workforce columns: {'md_nf_22': 'Total_Physicians'}


In [9]:
# Create Simplified Workforce Data
print("\n Creating simplified workforce dataset...")

# Extract basic columns
ahrf_cols = ['fips_st_cnty']

# Add any physician/provider columns we can find
physician_cols = [col for col in ahrf.columns if ('md_' in col.lower() or 'physician' in col.lower()) and '22' in col]
nurse_cols = [col for col in ahrf.columns if 'rn' in col.lower() and '22' in col]
dentist_cols = [col for col in ahrf.columns if 'dent' in col.lower() and '22' in col]

print(f"Found physician columns: {len(physician_cols)}")
print(f"Found nurse columns: {len(nurse_cols)}")
print(f"Found dentist columns: {len(dentist_cols)}")

# Use available columns
available_cols = ahrf_cols.copy()
if physician_cols:
    available_cols.extend(physician_cols[:5])  # Take first 5
if nurse_cols:
    available_cols.extend(nurse_cols[:3])      # Take first 3
if dentist_cols:
    available_cols.extend(dentist_cols[:2])    # Take first 2

print(f"Using columns: {available_cols[:10]}...")  # Show first 10

ahrf_subset = ahrf[available_cols].copy()
ahrf_subset['FIPS'] = ahrf_subset['fips_st_cnty'].astype(str).str.zfill(5)

# Create simplified provider count (sum of all numeric columns)
numeric_cols = available_cols[1:]  # Exclude fips_st_cnty
provider_counts = []

for _, row in ahrf_subset.iterrows():
    total_providers = 0
    for col in numeric_cols:
        val = pd.to_numeric(row[col], errors='coerce')
        if not pd.isna(val):
            total_providers += val
    provider_counts.append(max(1, total_providers))  # Ensure at least 1

ahrf_subset['Total_Healthcare_Providers'] = provider_counts

ahrf_clean = ahrf_subset[['FIPS', 'Total_Healthcare_Providers']]
print(f" AHRF cleaned: {len(ahrf_clean)} counties")
print("Sample AHRF data:")
print(ahrf_clean.head())



 Creating simplified workforce dataset...
Found physician columns: 813
Found nurse columns: 42
Found dentist columns: 38
Using columns: ['fips_st_cnty', 'md_nf_prim_care_pc_excl_rsdnt_22', 'md_nf_prim_care_pc_rsdnt_22', 'md_nf_fed_activ_22', 'md_nf_activ_22', 'md_nf_fed_22', 'aprn_npi_22', 'aprn_npi_mal_22', 'aprn_npi_fem_22', 'dent_npi_22']...
 AHRF cleaned: 3240 counties
Sample AHRF data:
    FIPS  Total_Healthcare_Providers
0  01001                       266.0
1  01003                      2472.0
2  01005                        89.0
3  01007                       201.0
4  01009                       126.0


In [10]:
# Merge All Data
print("\n Merging all datasets...")

# Clean geographic data
gdf_clean = gdf_counties.copy()
gdf_clean['FIPS'] = gdf_clean['GEOID'].astype(str).str.zfill(5)

if gdf_clean.crs != 'EPSG:4326':
    gdf_clean = gdf_clean.to_crs('EPSG:4326')

print(f"Geographic data prepared: {len(gdf_clean)} counties")

# Merge step by step
print("Merging ACS + AHRF...")
merged_data = pd.merge(acs_clean, ahrf_clean, on='FIPS', how='inner')
print(f"After ACS + AHRF merge: {len(merged_data)} counties")

print("Adding geographic data...")
final_gdf = pd.merge(gdf_clean, merged_data, on='FIPS', how='inner')
print(f" Final merged dataset: {len(final_gdf)} counties")

# Show sample of final data
print("\nSample of final merged data:")
sample_cols = ['NAME_y', 'STATEFP', 'Total_Pop', 'Pct_Minority', 'Total_Healthcare_Providers']
print(final_gdf[sample_cols].head())


 Merging all datasets...
Geographic data prepared: 3235 counties
Merging ACS + AHRF...
After ACS + AHRF merge: 3222 counties
Adding geographic data...
 Final merged dataset: 3222 counties

Sample of final merged data:
                         NAME_y STATEFP  Total_Pop  Pct_Minority  \
0       Cuming County, Nebraska      31       8976        8988.9   
1  Wahkiakum County, Washington      53       4573        4579.1   
2    De Baca County, New Mexico      35       1580        1602.3   
3    Lancaster County, Nebraska      31     323673      323680.2   
4     Nuckolls County, Nebraska      31       4089        4095.3   

   Total_Healthcare_Providers  
0                        42.0  
1                        19.0  
2                        13.0  
3                      4010.0  
4                        33.0  


In [11]:
# Calculate Basic Metrics
print("\n Calculating healthcare metrics...")

# Basic shortage score: population per provider
final_gdf['Population_Per_Provider'] = final_gdf['Total_Pop'] / final_gdf['Total_Healthcare_Providers']
final_gdf['Providers_Per_100k'] = (final_gdf['Total_Healthcare_Providers'] / final_gdf['Total_Pop']) * 100000

# Replace infinite values
final_gdf['Population_Per_Provider'] = final_gdf['Population_Per_Provider'].replace([np.inf, -np.inf], np.nan)
final_gdf['Providers_Per_100k'] = final_gdf['Providers_Per_100k'].replace([np.inf, -np.inf], np.nan)

# Create shortage score (0-100, higher = worse shortage)
max_pop_per_provider = final_gdf['Population_Per_Provider'].quantile(0.95)  # Use 95th percentile as max
final_gdf['Shortage_Score'] = (final_gdf['Population_Per_Provider'] / max_pop_per_provider * 100).clip(0, 100)

# Create equity index (simplified)
final_gdf['Equity_Index'] = 100 - (final_gdf['Pct_Minority'] * 0.8)  # Simplified equity measure
final_gdf['Equity_Index'] = final_gdf['Equity_Index'].clip(0, 100)

# Combined risk score
final_gdf['Combined_Risk_Score'] = (final_gdf['Shortage_Score'] * 0.6 + (100 - final_gdf['Equity_Index']) * 0.4)

# Add state names
state_map = {
    '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas', '06': 'California',
    '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware', '11': 'DC', '12': 'Florida',
    '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois', '18': 'Indiana',
    '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana', '23': 'Maine',
    '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota', '28': 'Mississippi',
    '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada', '33': 'New Hampshire',
    '34': 'New Jersey', '35': 'New Mexico', '36': 'New York', '37': 'North Carolina', '38': 'North Dakota',
    '39': 'Ohio', '40': 'Oklahoma', '41': 'Oregon', '42': 'Pennsylvania', '44': 'Rhode Island',
    '45': 'South Carolina', '46': 'South Dakota', '47': 'Tennessee', '48': 'Texas', '49': 'Utah',
    '50': 'Vermont', '51': 'Virginia', '53': 'Washington', '54': 'West Virginia', '55': 'Wisconsin', '56': 'Wyoming'
}
final_gdf['State'] = final_gdf['STATEFP'].map(state_map)

print(" Metrics calculated!")
print("\nKey Statistics:")
print(f"Average Shortage Score: {final_gdf['Shortage_Score'].mean():.1f}")
print(f"Average Equity Index: {final_gdf['Equity_Index'].mean():.1f}")
print(f"Average Combined Risk: {final_gdf['Combined_Risk_Score'].mean():.1f}")



 Calculating healthcare metrics...
 Metrics calculated!

Key Statistics:
Average Shortage Score: 32.6
Average Equity Index: 0.0
Average Combined Risk: 59.5


In [12]:
# Quick Visualization
print("\n6️⃣ Creating basic visualization...")

# Calculate centroids for mapping
final_gdf['lat'] = final_gdf.geometry.centroid.y
final_gdf['lon'] = final_gdf.geometry.centroid.x

# Create simple scatter map
fig = px.scatter_mapbox(
    final_gdf.head(200),  # Sample first 200 for speed
    lat='lat',
    lon='lon',
    color='Combined_Risk_Score',
    size='Total_Pop',
    hover_name='NAME_y',
    hover_data={
        'State': True,
        'Shortage_Score': ':.1f',
        'Equity_Index': ':.1f',
        'Pct_Minority': ':.1f',
        'lat': False,
        'lon': False
    },
    color_continuous_scale='RdYlBu_r',
    size_max=20,
    zoom=3,
    title='Healthcare Workforce Risk Map (Sample)',
    height=600
)

fig.update_layout(mapbox_style="open-street-map")
fig.show()

print("✅ Basic map created!")

# Summary Statistics
print("\n FINAL ANALYSIS SUMMARY")
print("="*50)
print(f"Dataset: {len(final_gdf)} counties analyzed")
print(f"States covered: {final_gdf['State'].nunique()}")
print(f"Total population: {final_gdf['Total_Pop'].sum():,}")

print(f"\n Healthcare Workforce:")
print(f"Total providers: {final_gdf['Total_Healthcare_Providers'].sum():,}")
print(f"Avg providers per 100k: {final_gdf['Providers_Per_100k'].mean():.1f}")

print(f"\n Risk Assessment:")
high_risk = (final_gdf['Combined_Risk_Score'] > 70).sum()
print(f"High risk counties (>70): {high_risk} ({high_risk/len(final_gdf)*100:.1f}%)")




6️⃣ Creating basic visualization...


✅ Basic map created!

 FINAL ANALYSIS SUMMARY
Dataset: 3222 counties analyzed
States covered: 51
Total population: 335,642,425

 Healthcare Workforce:
Total providers: 4,788,112.0
Avg providers per 100k: 750.1

 Risk Assessment:
High risk counties (>70): 541 (16.8%)


In [13]:

print("🔧 FIXING DATA ISSUES")
print("="*50)

# =============================================
# FIX 1: Correct the Minority Percentage Calculation
# =============================================
print("Fixing minority percentage calculation...")

# The issue: Asian column has population instead of percentage
# Let's recalculate properly

# Check what's in the Asian column
print("Sample Asian values:", final_gdf['Pct_Asian'].head())

# Fix: Use only Black and Hispanic percentages for now
final_gdf['Pct_Minority_Fixed'] = (
    final_gdf['Pct_Black'].fillna(0) +
    final_gdf['Pct_Hispanic'].fillna(0)
).clip(0, 100)  # Cap at 100%

# Recalculate Equity Index
final_gdf['Equity_Index_Fixed'] = (100 - final_gdf['Pct_Minority_Fixed'] * 0.8).clip(0, 100)

# Recalculate Combined Risk Score
final_gdf['Combined_Risk_Score_Fixed'] = (
    final_gdf['Shortage_Score'] * 0.6 +
    (100 - final_gdf['Equity_Index_Fixed']) * 0.4
)

print("✅ Data fixed!")
print(f"New Average Minority %: {final_gdf['Pct_Minority_Fixed'].mean():.1f}%")
print(f"New Average Equity Index: {final_gdf['Equity_Index_Fixed'].mean():.1f}")
print(f"New Average Combined Risk: {final_gdf['Combined_Risk_Score_Fixed'].mean():.1f}")

# =============================================
# ADVANCED VISUALIZATION 1: Interactive Risk Map
# =============================================
print("\n📊 CREATING ADVANCED VISUALIZATIONS")
print("="*50)

# Create risk categories
final_gdf['Risk_Category'] = pd.cut(
    final_gdf['Combined_Risk_Score_Fixed'],
    bins=[0, 25, 50, 75, 100],
    labels=['Low Risk', 'Moderate Risk', 'High Risk', 'Critical Risk']
)

# Enhanced interactive map
fig_map = px.scatter_mapbox(
    final_gdf,
    lat='lat',
    lon='lon',
    color='Combined_Risk_Score_Fixed',
    size='Total_Pop',
    hover_name='NAME_y',
    hover_data={
        'State': True,
        'Shortage_Score': ':.1f',
        'Equity_Index_Fixed': ':.1f',
        'Pct_Minority_Fixed': ':.1f',
        'Total_Healthcare_Providers': ':,',
        'Providers_Per_100k': ':.1f',
        'Risk_Category': True,
        'lat': False,
        'lon': False
    },
    color_continuous_scale='RdYlBu_r',
    size_max=25,
    zoom=3.5,
    title='🏥 Healthcare Workforce Risk Assessment Map<br><sub>🔴 Red = Higher Risk | 🔵 Blue = Lower Risk | Size = Population</sub>',
    height=700,
    width=1000
)

fig_map.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=39.5, lon=-98.35)),
    font=dict(size=12),
    title_font_size=16,
    coloraxis_colorbar=dict(
        title="Combined Risk Score<br>(0=Low, 100=Critical)"
    )
)

fig_map.show()

# =============================================
# ADVANCED VISUALIZATION 2: Multi-Panel Dashboard
# =============================================
print("Creating comprehensive dashboard...")

fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '🏥 Healthcare Provider Distribution',
        '🎯 Risk vs Equity Analysis',
        '📊 State-Level Risk Rankings',
        '🍕 Risk Category Distribution'
    ),
    specs=[
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"type": "domain"}]
    ]
)

# 1. Provider distribution histogram
fig_dashboard.add_trace(
    go.Histogram(
        x=final_gdf['Providers_Per_100k'],
        nbinsx=30,
        marker_color='skyblue',
        name='Provider Density',
        opacity=0.7
    ),
    row=1, col=1
)

# 2. Risk vs Equity scatter
fig_dashboard.add_trace(
    go.Scatter(
        x=final_gdf['Equity_Index_Fixed'],
        y=final_gdf['Shortage_Score'],
        mode='markers',
        marker=dict(
            color=final_gdf['Pct_Minority_Fixed'],
            colorscale='Viridis',
            size=8,
            opacity=0.6,
            colorbar=dict(title="Minority %", x=0.47)
        ),
        text=final_gdf['NAME_y'],
        name='Counties',
        hovertemplate='<b>%{text}</b><br>Equity: %{x:.1f}<br>Shortage: %{y:.1f}<extra></extra>'
    ),
    row=1, col=2
)

# Add quadrant lines
median_equity = final_gdf['Equity_Index_Fixed'].median()
median_shortage = final_gdf['Shortage_Score'].median()

fig_dashboard.add_hline(
    y=median_shortage, line_dash="dash", line_color="red",
    annotation_text=f"Median Shortage ({median_shortage:.1f})",
    row=1, col=2
)
fig_dashboard.add_vline(
    x=median_equity, line_dash="dash", line_color="red",
    annotation_text=f"Median Equity ({median_equity:.1f})",
    row=1, col=2
)

# 3. State rankings
state_risk = final_gdf.groupby('State')['Combined_Risk_Score_Fixed'].mean().sort_values(ascending=False).head(15)

fig_dashboard.add_trace(
    go.Bar(
        x=state_risk.values,
        y=state_risk.index,
        orientation='h',
        marker_color='red',
        name='State Risk Scores',
        opacity=0.8
    ),
    row=2, col=1
)

# 4. Risk category pie chart
risk_counts = final_gdf['Risk_Category'].value_counts()
colors = ['green', 'yellow', 'orange', 'red']

fig_dashboard.add_trace(
    go.Pie(
        labels=risk_counts.index,
        values=risk_counts.values,
        marker_colors=colors,
        name='Risk Distribution',
        textinfo='label+percent',
        textfont_size=12
    ),
    row=2, col=2
)

fig_dashboard.update_layout(
    height=800,
    title_text="🏥 Healthcare Workforce Analysis Dashboard",
    showlegend=False,
    font=dict(size=11)
)

# Update axis labels
fig_dashboard.update_xaxes(title_text="Providers per 100k Population", row=1, col=1)
fig_dashboard.update_yaxes(title_text="Number of Counties", row=1, col=1)

fig_dashboard.update_xaxes(title_text="Equity Index (Higher = Better)", row=1, col=2)
fig_dashboard.update_yaxes(title_text="Shortage Score (Higher = Worse)", row=1, col=2)

fig_dashboard.update_xaxes(title_text="Average Risk Score", row=2, col=1)
fig_dashboard.update_yaxes(title_text="States", row=2, col=1)

fig_dashboard.show()

# =============================================
# ADVANCED VISUALIZATION 3: Heatmap Analysis
# =============================================
print("Creating correlation heatmap...")

# Select key variables for correlation
heatmap_vars = [
    'Shortage_Score',
    'Equity_Index_Fixed',
    'Combined_Risk_Score_Fixed',
    'Pct_Minority_Fixed',
    'Total_Pop',
    'Providers_Per_100k',
    'Population_Per_Provider'
]

# Calculate correlation matrix
corr_matrix = final_gdf[heatmap_vars].corr()

# Create heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=[var.replace('_', ' ').replace('Fixed', '') for var in corr_matrix.columns],
    y=[var.replace('_', ' ').replace('Fixed', '') for var in corr_matrix.index],
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values.round(3),
    texttemplate="%{text}",
    textfont={"size": 12},
    hoverongaps=False,
    colorbar=dict(title="Correlation<br>Coefficient")
))

fig_heatmap.update_layout(
    title='🔥 Healthcare Metrics Correlation Heatmap<br><sub>🔴 Red = Positive Correlation | 🔵 Blue = Negative Correlation</sub>',
    height=600,
    width=700,
    font=dict(size=12)
)

fig_heatmap.show()

print("\n KEY INSIGHTS")
print("="*60)

# Calculate key statistics
total_counties = len(final_gdf)
critical_risk = (final_gdf['Risk_Category'] == 'Critical Risk').sum()
high_risk = (final_gdf['Risk_Category'] == 'High Risk').sum()

# Top risk counties
top_risk_counties = final_gdf.nlargest(10, 'Combined_Risk_Score_Fixed')[
    ['NAME_y', 'State', 'Combined_Risk_Score_Fixed', 'Shortage_Score', 'Pct_Minority_Fixed']
]

# Rural vs Urban analysis (using population as proxy)
final_gdf['County_Type'] = pd.cut(
    final_gdf['Total_Pop'],
    bins=[0, 25000, 100000, np.inf],
    labels=['Rural', 'Suburban', 'Urban']
)

type_analysis = final_gdf.groupby('County_Type').agg({
    'Combined_Risk_Score_Fixed': 'mean',
    'Shortage_Score': 'mean',
    'Pct_Minority_Fixed': 'mean'
}).round(1)

print(f" EXECUTIVE SUMMARY:")
print(f"   • Total Counties Analyzed: {total_counties:,}")
print(f"   • Critical Risk Counties: {critical_risk:,} ({critical_risk/total_counties*100:.1f}%)")
print(f"   • High + Critical Risk: {high_risk + critical_risk:,} ({(high_risk + critical_risk)/total_counties*100:.1f}%)")

print(f"\nKEY FINDINGS:")
print(f"   • Average Shortage Score: {final_gdf['Shortage_Score'].mean():.1f}/100")
print(f"   • Average Equity Index: {final_gdf['Equity_Index_Fixed'].mean():.1f}/100")
print(f"   • Shortage-Minority Correlation: r = {final_gdf['Shortage_Score'].corr(final_gdf['Pct_Minority_Fixed']):.3f}")

print(f"\n COUNTY TYPE ANALYSIS:")
for county_type in type_analysis.index:
    stats = type_analysis.loc[county_type]
    print(f"   • {county_type}: Risk={stats['Combined_Risk_Score_Fixed']:.1f}, Shortage={stats['Shortage_Score']:.1f}")

print(f"\n TOP 5 HIGHEST RISK COUNTIES:")
for i, (_, county) in enumerate(top_risk_counties.head().iterrows(), 1):
    print(f"   {i}. {county['NAME_y']}, {county['State']}")
    print(f"      Risk: {county['Combined_Risk_Score_Fixed']:.1f} | Shortage: {county['Shortage_Score']:.1f} | Minority: {county['Pct_Minority_Fixed']:.1f}%")



print(f"\n EXPORTING RESULTS...")

# Export main dataset
export_df = final_gdf.drop('geometry', axis=1)
export_df.to_csv('healthcare_workforce_final_analysis.csv', index=False)

# Export top risk counties
top_risk_counties.to_csv('top_risk_counties_for_intervention.csv', index=False)

# Export state summary
state_summary = final_gdf.groupby('State').agg({
    'Combined_Risk_Score_Fixed': 'mean',
    'Shortage_Score': 'mean',
    'Equity_Index_Fixed': 'mean',
    'Total_Pop': 'sum',
    'Total_Healthcare_Providers': 'sum'
}).round(1).sort_values('Combined_Risk_Score_Fixed', ascending=False)

state_summary.to_csv('state_level_healthcare_summary.csv')

print(f" Files exported:")
print(f"   • healthcare_workforce_final_analysis.csv")
print(f"   • top_risk_counties_for_intervention.csv")
print(f"   • state_level_healthcare_summary.csv")



🔧 FIXING DATA ISSUES
Fixing minority percentage calculation...
Sample Asian values: 0      8976
1      4573
2      1580
3    323673
4      4089
Name: Pct_Asian, dtype: int64
✅ Data fixed!
New Average Minority %: 10.4%
New Average Equity Index: 91.7
New Average Combined Risk: 22.9

📊 CREATING ADVANCED VISUALIZATIONS


Creating comprehensive dashboard...


Creating correlation heatmap...



 KEY INSIGHTS
 EXECUTIVE SUMMARY:
   • Total Counties Analyzed: 3,222
   • Critical Risk Counties: 11 (0.3%)
   • High + Critical Risk: 260 (8.1%)

KEY FINDINGS:
   • Average Shortage Score: 32.6/100
   • Average Equity Index: 91.7/100
   • Shortage-Minority Correlation: r = 0.078

 COUNTY TYPE ANALYSIS:
   • Rural: Risk=29.8, Shortage=43.5
   • Suburban: Risk=18.7, Shortage=26.6
   • Urban: Risk=12.4, Shortage=15.0

 TOP 5 HIGHEST RISK COUNTIES:
   1. Kusilvak Census Area, Alaska, Alaska
      Risk: 90.5 | Shortage: 100.0 | Minority: 95.4%
   2. Zapata County, Texas, Texas
      Risk: 88.6 | Shortage: 100.0 | Minority: 89.3%
   3. Brooks County, Texas, Texas
      Risk: 86.4 | Shortage: 100.0 | Minority: 82.5%
   4. Lake and Peninsula Borough, Alaska, Alaska
      Risk: 85.4 | Shortage: 100.0 | Minority: 79.5%
   5. Duval County, Texas, Texas
      Risk: 85.1 | Shortage: 100.0 | Minority: 78.4%

 EXPORTING RESULTS...
 Files exported:
   • healthcare_workforce_final_analysis.csv
   • 